## Spark I

Primeros pasos con Apache Spark


**¿Qué es spark?**

> Apache spark es un motor unificado de cálculo en memoria y un conjunto de bibliotecas para procesamiento paralelo y distribuido de datos en clústeres de ordenadores
> 

**¿Cómo se puede usar Spark?** 

1. Desde un API de DataFrames
2. Lanzando consultas SQL 
3. Desde una instrucción de la API 
4. Desde un consulta de tarea de grafos 

**¿Qué es el echo de que sea en memoria?**

> Todos los cálculos se llevan en memoria y solo se escriben resultados en disco sí el usuario lo especifica o que la operación requiere mover datos entre nodos (shuffle)
> 

**¿Qué ventajas tiene?**

- Consiste en APi más intuitiva que MapReduce
- Abstrae todos los detalles de comunicación en red.
- Opera similar a transacciones SQL (tablas distribuidas)
- Ofrece para  distintos lenguajes: Java, Scala y Python
- Pyspark es el paquete de python

**¿Cómo se componen Spark?**
![Untitled](./imgs/Untitled.png)


- Spark core: compoenente principal, almacena estructuras de datos fundamentales para trabajar con spark e.g RDD
- Los tres componentes  inferiores, permiten gestionar recursos del clúster, maquinas, CPU y RAM
- Spark SQL: API para manejar tablas distribuidas, estructuradas con nombre y tipo, DataFrames.
- Spark Streaming: Procesamiento de flujos de datos en tiempo real
- Spark MLlib: Implementaciones distribuidas de algoritmos ML
- Spark GraphX: Representa un grafo como una pareja de DataFrames, con los nodos y los arcos.

## Arquitectura de Spark

<aside>
🎯 Al escribir  na aplicación en spark es una aplicación **secuencial.** No paralela, que es en una máquina interna o externa al clúster.  El programa driver puede conectarse al servidor de clúster.

</aside>

 

![Untitled](imgs/Untitled%201.png)

Al ejecutar usamos un sparkSession, 

**¿Qué es el sparkSession?**

Es el encargado de establecer la conexión con el gestor de clúster para poder enviar tareas a los workers. 

 Al crear un sparkSession es necesario:

- Indicar el número de nodos
- RAM
- Número de cores físicos

Esto para cada nodo. Estos recursos se llaman *executor,* 

**¿Qué es un *executor*?**

<aside>
🎯 Un *executor: es un proceso de la JVM que se ejecuta en el nodo y ocupa los recursos establecidos en el sparkSession.*

</aside>

Un ejecutor es creado por el gestor del clúster  cuando arranca nuestra aplicación de Spark y muere cuando la aplicación finaliza 

> Cada ejecutor queda preparado para ejecutar *tareas* de Spark, que es la unidad mínima de ejecución de trabajos. Cada tarea requiere un *core* libre para ejecutarse, por lo que, si un *executor* tiene reservados cuatro *cores*, podrá ejecutar cuatro tareas en paralelo.
> 

<aside>
🎯 Cuando el flujo del programa llega a ciertas funciones especificas de Spark que desencadenan ejecución distribuida.

</aside>

### Resilient Distributed Datasets

¿Qué son los RDD?

- Es una abstracción fundamental de Spark
- Es una colección no ordenada de objetos (dag)
- Vive en la RAM de los Nodos de un clúster

Las colecciones está dividida en particiones, las cuales están en la RAM de los nodos 

1. Resilient: Resistente, Adaptable. Es posible reconstruir un RDD usando los DAG 
2. Distributed: Particiones de están distribuidos en la RAM de los nodos, al no estar ordenado no es posible acceder por indice.  
3. Dataset: La colección representa un conjunto de datos que estamos procesando de forma paralela. 

<aside>
⚠️ **no hay replicación de cada partición**.

</aside>

![Untitled](imgs/Untitled%202.png)

es posible reconstruir las particiones que estuvieran en ese momento en su memoria principal gracias al DAG. 

**¿Qué es la inmutabilidad?**

Un RDD no puede modificarse una vez creado. Lo que hacemos es aplicar transformaciones a los RDD para obtener otros datos.  Las peticiones se ejecutan en paralelo sobre todas las paticiones del RDD de manera transparente, lo que da lugar a un nuevo RDD. 

> **Ejemplo**: dado un RDD de números reales, para multiplicar cada elemento por
dos, aplicamos una transformación que actúa en cada elemento y lo multiplica
por dos. Spark lleva nuestro código de la transformación (lo serializa y lo envía
por la red) a cada uno de los nodos del clúster donde haya particiones de ese
RDD y lo ejecuta en ellos para que actúe en cada elemento de esa partición.
Todo de manera transparente al programador.
> 

### Partición

- Una partición es un subconjunto de los objetos presentes en un RDD que residen en un mismo nodo en un clúster distribuido.
- Las particiones son la unidad de datos mínima en la cual se ejecuta una tarea de transformación de manera independiente a las demás particiones.
- Es recomendable tener al menos tantas particiones como cores físicos (procesadores) disponibles en el clúster para aprovechar al máximo los recursos de hardware.
- Idealmente, cada core debería tener una partición asignada para garantizar que todos los cores estén ocupados incluso si solo se está utilizando la aplicación en el clúster.
- Lo habitual es tener muchas particiones de un mismo RDD en cada nodo, en un número superior al de procesadores disponibles en el nodo.
- Se recomienda que cada RDD esté dividido en un número de particiones que sea entre el doble y el triple del número de procesadores del clúster. Esto permite una mejor distribución de la carga de trabajo y aprovechamiento de los recursos.

<aside>
⚠️ Es recomendable usar DataFrames en lugar de RDD, el motor de spark llamado Catalyst hace optimizaciones en el código de los DataFrames

</aside>

### Transformaciones y acciones

**¿Qué son las transformaciones?**

Las transformaciones se ejecutan en un RDD que genera uno nuevo, son ******lazy,****** lo que hace spark es agregar la transformación al DAG y ejecutarla hasta que encuentre una acción. 

<aside>
⚠️ El **DAG** es el grao que permite hacer la trazabilidad de las operaciones y tener la *resiliency*

</aside>

- Sí la transformación no requiere un shuffle (movimiento de datos), es llamado **************narrow************** y

**¿Qué son las acciones?**

Recibe un RDD y calcula un resultado (integers, doubles etc) y lo devuelve al driver (programa principal)

<aside>
⚠️ El tipo de dato devuelto NO ES UN RDD, es un tipo nativo del lenguaje que estemos usando.

</aside>

Algunas puntos importantes de las acciones:

- El resultado cae en la memoria de la maquina driver
- Una acción desencadena la ejecución de las transformaciones intermedias y la materialización de los nuevos RDD
- Una vez creados los RDD se aplican las transformaciones que toque, según indica el DAG
- Se libera el RDD (no permanecen en memoria RAM )
- Un RDD cacheado permanece materializado en la RAM de los nodos y no es necesario recalcularlo después de que se haya materializado la primera vez.
- Cachear un RDD ayuda a no reconstruirlo, la reconstrucción de un RDD comienza con el DAG, acceso a los datos  y aplicar las transformaciones.

### Transformaciones más habituales

1. map: recibe una función que se ejecuta a cada elemento del RDD, retorna un nuevo RDD con los elementos transformados.
2. flatMap; recibe una función que se ejecuta por cada uno de los elementos y regresa un vector de valroes para cada elemento
3. filter: Filtra un RDD, regresa True si el elemento debe ser incluido en el nuevo RDD
4. sample: Genera una muestra aleatoria del RDD de tamaño especifica  
5. union: La unión de dos RDD parados como parámetro 
6. intersection: devuelve la intersección de dos RDD, los elementos que están presentes en ambos 
7. distinct: Quita los elementos repetidos  

**Transformaciones específicas para un PairRDD, es decir, RDD de pares (clave, valor):**

1. groupByKey: Se agrupan los elementos del RDD según su clave. Todos los elementos con la misma clave se agrupan juntos.
2. reduceByKey:  Se agrupan los elementos por clave, se pasa una función que recibe dos parámetros y retornar uno. 
3. sortByKey: Ordena los elementos del RDD por clave 
4. join: Combina dos RDD de tal modo que se junten los elementos que tienen la
misma clave.

<aside>
⚠️ En resumen, la propiedad conmutativa implica que el orden de las operaciones no afecta el resultado, mientras que la propiedad asociativa implica que el resultado es independiente de cómo se agrupen los operandos o los valores intermedios. Ambas propiedades son fundamentales en el procesamiento de RDDs y garantizan resultados consistentes y predecibles en operaciones algebraicas.

</aside>

### Acciones más habituales en RDD

<aside>
🎯 Todas las acciones llevan resultados a los drivers. por lo que estos tienen que caber en la memoria del proceso ******driver******

</aside>

1. reduce: 
2. collect: devuelve todos los elementos contenidos en el RDD como una colección, lanza una ecepción cuando la lista es muy grande y no cabe en la RAM
3. count: Número de elementos contenidos en el RDD
4. take: Devuelve los n primeros elementos. No hay garantías de ordenación. 
5. first: Retorna el primer elemento del RDD
6. takeSample: Devuelve n elemento aleatorios del RDD
7. takeOrdered: devuelve los n primeros elementos del RDD tras haber realizado
una ordenación de todos los elementos contenidos en el mismo.
8. countByKey: cuenta el número de elementos en el RDD para cada clave
diferente
9. saveAsTextFile: : guarda los contenidos del RDD en un fichero de texto.

![Untitled](imgs/Untitled%203.png)

```python
func_multiplicar = lambda x: (x, 3*x) # función que devuelve una tupla
A = sc.parallelize([5.0, 3.2, 1.1, -2.4, # distribuimos la lista como
 8.9, 4.4, 3.7, 9.1], 3) # un RDD de 3 particiones
# sc se refiere a un SparkContext que realiza la conexión 
# con el gestor de clúster.

B = A.map(func_multiplicar)
B = B.reduceByKey(lambda v1, v2: v1+v2)
C = sc.parallelize([(5.0, 1.0), (1.1, -3)], 4) # lista a RDD de 4 part
D = C.map(lambda tuple: (tuple[0], 2*tuple[1]))
E = D.filter(lambda tuple: tuple[1] > 1)
F = E.join(B)
resultado = F.take(3)

```

### Jobs, stages y tasks

<aside>
🎯 Un job (trabajo) de Spark es todo el procesamiento necesario para llevar a
cabo una acción del usuario.

</aside>

<aside>
🎯 Un stage es todo el procesamiento que puede llevarse a cabo sin mover datos
entre nodos

</aside>

<aside>
🎯 Una tarea de Spark es el procesamiento aplicado por un core físico (CPU) a
una partición de un RDD.

</aside>

![Untitled](imgs/Untitled%204.png)

### Ejemplo de Spark RDD

In [ ]:
from pyspark.sql import SparkSession

Crear una aplicación de 

In [2]:
spark = SparkSession.builder.appName("Spark_1").getOrCreate()

In [3]:
sparkContext = spark.sparkContext

En Spark existe diferentes formas de leer un archivo, aquí se muestra una forma "artesanal" de procesar un archivo CSV

In [ ]:
flights = sparkContext.textFile('./data/flights.csv')

In [6]:
header_line = flights.first()
flights_lines = flights.filter(lambda line: line != header_line)

Leer la primera línea para después filtrar el resto del archivo.

En Spark existen dos terminos importantes:
- Acciones: Recibe un RDD y calcula un resultado (integers, doubles etc) y lo devuelve al driver (programa principal)
- Transformaciones:Las transformaciones se ejecutan en un RDD que genera uno nuevo, son lazy, lo que hace spark es agregar la transformación al DAG y ejecutarla hasta que encuentre una acción. 

flights_lines.take(5)

<code>take(n)</code> es una acción que devuelve los n primeros elementos. No hay garantías de ordenación. 

In [ ]:
flights_rdd = flights_lines.map(lambda line: line.split(',')).map(lambda fields_list:           
                                     (int(fields_list[0]),
                                      int(fields_list[1]),
                                      int(fields_list[2]),
                                      fields_list[3],
                                      fields_list[4]))

El código anterior regresa un nuevo RDD con los campos "convertidos" a integers y strings 

In [12]:
flights_rdd.take(10)

[(2013, 1, 1, 'EWR', 'IAH'),
 (2013, 1, 1, 'LGA', 'IAH'),
 (2013, 1, 1, 'JFK', 'MIA'),
 (2013, 1, 1, 'JFK', 'BQN'),
 (2013, 1, 1, 'LGA', 'ATL'),
 (2013, 1, 1, 'EWR', 'ORD'),
 (2013, 1, 1, 'EWR', 'FLL'),
 (2013, 1, 1, 'LGA', 'IAD'),
 (2013, 1, 1, 'JFK', 'MCO'),
 (2013, 1, 1, 'LGA', 'ORD')]

La transformación <code>map(function)</code>, recibe una función que se ejecuta a cada elemento del RDD, retorna un nuevo RDD con los elementos transformados.

En la siguiente función lambda crea un nuevo PairRDD clave-valor,  cuya pisición 0 de la tupla es la clave y el valor es posición 1 de la tupla. 


En este caso se crea una tupla con la clave de destino con valor 1

In [13]:
flights_rdd_count = flights_rdd.map(lambda line: (line[4],1))

In [14]:
flights_rdd_count.take(5)

[('IAH', 1), ('IAH', 1), ('MIA', 1), ('BQN', 1), ('ATL', 1)]

Para contar cuántos vuelos tenie como destino cada uno de los destinos en el fichero flights.csv Para ello, podemos utilizar el pairRDD previo, agrupar por destino y sumar los valores de cada clave:

In [19]:
flights_dest_rdd_groupped = flights_rdd_count.groupByKey().map(
    lambda line: (line[0],sum( line[1] ))
)

In [20]:
flights_dest_rdd_groupped.take(10)

[('IAH', 7198),
 ('MIA', 11728),
 ('BQN', 896),
 ('ORD', 17283),
 ('IAD', 5700),
 ('MCO', 14082),
 ('TPA', 7466),
 ('DFW', 8738),
 ('BOS', 15508),
 ('MSP', 7185)]